# Project proposal



##GriSPy - Grid Search in Python

GriSPy is a regular grid search algorithm for quick nearest-neighbor lookup.

This class indexes a set of k-dimensional points in a regular grid providing a fast aproach for nearest neighbors queries. Optional periodic boundary conditions can be provided for each axis individually.

###GriSPy has the following queries implemented:

> **`bubble_neighbors`**: find neighbors within a given radius. A different radius for each centre can be provided.
>
> **`shell_neighbors`**: find neighbors within given lower and upper radius. Different lower and upper radius can be provided for each centre.
>
> **`nearest_neighbors`**: find the nth nearest neighbors for each centre.

###And the following methods are available:

> **`set_periodicity`**: define the periodicity conditions.
>
> **`save_grid`**: save the grid for future use.

## Use

### Build the grid

Build the grid with the **data**, the set of k-dimensional points where neighbors are sought:


```python
gsp = GriSPy(data, periodic=periodic)
```

**data** is a numpy array of shape (N, k)

**periodic** (Optional) is a dictionary indicating if the data domain is periodic in some or all its dimensions.
Example, periodic = { 0: (0, 360), 1: None}.

The periodicity can be set when the grid is created or after.

```python
gsp = GriSPy(data)
gsp.set_periodicity(periodic)
```

### Query for neighbors within upper_radii

```python
bubble_dist, bubble_ind = gsp.bubble_neighbors(centres, distance_upper_bound=upper_radii)
``` 

### Query for neighbors in a shell within lower_radii and upper_radii

```python
shell_dist, shell_ind = gsp.shell_neighbors(centres,
 distance_lower_bound=lower_radii, 
 distance_upper_bound=upper_radii)
```

### Query for nth nearest neighbors 

```python
near_dist, near_ind = gsp.nearest_neighbors(centres, n=n_nearest)
```

### Inputs

**centres** is a numpy array of M centres with same dimesion that **data**

**lower_radii** and **upper_radii** are integers if the search radii are the same for all centers or array_like with M different radii

**n_nearest** is an integer

### Outputs
All these methods return the distance (**_dist**) of each neighbor to their respective center, and the position or index (**_ind**) of each neighbor within the **data** array.

# Estimation

In [0]:
import numpy as np
import pert

Task dictionary, every key is a task and every body is a tuple with the `(o, p, m)`

Where:

- optimistic time (O): the minimum possible time required to accomplish a
  task, assuming everything proceeds better than is normally expected
- pessimistic time (P): the maximum possible time required to accomplish a
  task, assuming everything goes wrong (but excluding major catastrophes).
- most likely time (M): the best estimate of the time required to
  accomplish a task, assuming everything proceeds as normal.

All times are expresed in **hours**

In [0]:
fee = 200

tasks = {
    "Reescritura 2.7 -> 3.7": (10., 20., 15.),
    "Documentación": (5., 10., 8.),
    "Testing": (30., 60., 50.),
}

### Expected time by taks

the best estimate of the time required to accomplish a
task, accounting for the fact that things don't always proceed as normal
(the implication being that the expected time is the average time the
task would require if the task were repeated on a number of occasions over
an extended period of time).

```
TE = (O + 4M + P) / 6
```

In [3]:
for k, v in tasks.items():
    print(k.rjust(30), "-", "%.2f hs" % pert.expect(*v))

        Reescritura 2.7 -> 3.7 - 15.00 hs
                 Documentación - 7.83 hs
                       Testing - 48.33 hs


### Global Estimation

Create a estimation for a set of values. This return three range of values with probablilities of 68.2%, 95.4%, 99.7%

In [0]:
optimists = [v[0] for v in tasks.values()]
pesimists = [v[1] for v in tasks.values()]
most_likely = [v[2] for v in tasks.values()]

In [5]:
probs = pert.estimate(optimists, pesimists, most_likely)

print(
    "With {0}% of probability the project will be ended in between {1:.2f} and {2:.2f} hours".format("68.2", *probs[0]))
print(
    "With {0}% of probability the project will be ended in between {1:.2f} and {2:.2f} hours".format("95.4", *probs[1]))
print(
    "With {0}% of probability the project will be ended in between {1:.2f} and {2:.2f} hours".format("99.7", *probs[2]))

With 68.2% of probability the project will be ended in between 63.67 and 86.17 hours
With 95.4% of probability the project will be ended in between 56.17 and 86.17 hours
With 99.7% of probability the project will be ended in between 48.67 and 93.67 hours


### Conclusion

In [6]:
limits = (probs[0][0], probs[2][1])
costs = (probs[0][0] * fee, probs[2][1] * fee)
print(
    "The project will be made in a range between of {0:.2f} (worst best) and {1:.2f} (best worst) hours".format(*limits))
print("In money the project will cost in between {0:.2f} usd and {1:.2f} usd".format(*costs))

The project will be made in a range between of 63.67 (worst best) and 93.67 (best worst) hours
In money the project will cost in between 12733.33 usd and 18733.33 usd


In [7]:
import datetime; datetime.datetime.now().isoformat()

'2019-09-10T22:25:14.089899'

#An estimation is not a deadline